In [1]:
from __future__ import annotations

import pandas as pd
import plotly.graph_objects as go

from data import DataManager
from utils import compute_sr_boxes, plot_sr_boxes

# ------------------------------------------------------------------
# Load 15m OHLCV for the past 30 days from OKX
# ------------------------------------------------------------------
manager = DataManager()
end_time = pd.Timestamp.utcnow().floor("min")
start_time = end_time - pd.Timedelta(days=30)

data = manager.get_klines(
    symbol="BTCUSDT",
    timeframe="15m",
    start=start_time,
    end=end_time,
    source="okx",
)


In [3]:
from utils.support_resistance import _pivot_series

ht_close = data.resample("4H").last()["close"].dropna()
pivot = _pivot_series(ht_close, length=20, mode="high")
print("pivot highs: ", pivot.dropna().head())



pivot highs:  Series([], Freq: 4h, dtype: float64)


C:\Users\uncczhangtao\AppData\Local\Temp\ipykernel_1199244\3536408189.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  ht_close = data.resample("4H").last()["close"].dropna()


In [4]:
data

,open,high,low,close,volume
timestamp,,,,,
2025-10-25 22:30:00+00:00,111511.9,111619.0,111511.9,111606.4,12.681505
2025-10-25 22:45:00+00:00,111606.7,111694.3,111600.0,111630.2,15.076149
2025-10-25 23:00:00+00:00,111630.2,111675.0,111553.0,111649.9,22.241105
2025-10-25 23:15:00+00:00,111649.9,111665.3,111563.5,111641.9,13.384339
2025-10-25 23:30:00+00:00,111641.9,111641.9,111532.5,111558.9,13.358590
...,...,...,...,...,...
2025-11-09 21:15:00+00:00,104385.2,104646.0,104385.1,104639.7,17.352930
2025-11-09 21:30:00+00:00,104641.0,104848.3,104558.0,104774.2,27.484189
2025-11-09 21:45:00+00:00,104776.0,104787.2,104318.1,104532.4,56.374643


In [7]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd

# 假设 ohlcv_15m 已存在，并且 index 是 UTC 时间
htf = data.resample("4H").agg(
    {
        "open": "first",
        "high": "max",
        "low": "min",
        "close": "last",
        "volume": "sum",
    }
).dropna()

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.03,
                    row_heights=[0.7, 0.3])

fig.add_trace(
    go.Candlestick(
        x=htf.index,
        open=htf["open"],
        high=htf["high"],
        low=htf["low"],
        close=htf["close"],
        name="4H Candles",
    ),
    row=1, col=1,
)

colors = ["rgba(39,174,96,0.7)" if c >= o else "rgba(192,57,43,0.7)"
          for c, o in zip(htf["close"], htf["open"])]
fig.add_trace(
    go.Bar(
        x=htf.index,
        y=htf["volume"],
        marker_color=colors,
        name="Volume",
    ),
    row=2, col=1,
)

fig.update_layout(
    title="BTCUSDT 4H Candles with Volume",
    xaxis2_title="Time",
    yaxis_title="Price",
    yaxis2_title="Volume",
    template="plotly_white",
    height=720,
)

fig.show()

C:\Users\uncczhangtao\AppData\Local\Temp\ipykernel_1199244\562717614.py:6: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



In [5]:
boxes, events = compute_sr_boxes(data, target_timeframe="4h")
fig = plot_sr_boxes(data, boxes, events, title="BTCUSDT 15m + 4H SR")
fig.show()